# R3T2

# openning Data

In [7]:
using GLPK
using SparseArrays
using JuMP
using COSMO
using LinearAlgebra
using MAT

In [8]:
file = matopen("R3T2.mat")
varnames = names(file)

3-element Array{String,1}:
 "L"
 "S"
 "U"

In [9]:
S = read(file, "S")
L = read(file, "L")
U = read(file, "U");

In [10]:
(m,n)=size(S)

(2172, 4456)

In [11]:
(r,c) = size(L)

(4456, 50)

In [12]:
SVzeros = zeros(m,c);

# constructing model

In [13]:
function zero_rows(V)
    indexes= zeros(Int64,size(V)[1],1)
    k=1
    for i in 1:size(V)[1]
        if sum(V[i,:].*V[i,:])<1e-10
            indexes[k]= i
            k+=1
        end
    end
    return indexes[1:k-1]
end

zero_rows (generic function with 1 method)

In [14]:
function solv(indexes)
    answer= zeros(r,c)
    col_zeros= zeros(m,1)
    for j in 1:c
        if j%500==0
            print(j)
        end
        model = Model(GLPK.Optimizer)
        @variable(model, v[1:r])
        @variable(model, w[1:r])
        @constraint(model, S*v .== col_zeros)
        @constraint(model,v.>=L[:,j])
        @constraint(model,U[:,j].>=v)
        @constraint(model,w.>=v)
        @constraint(model,w.>=-v)
        for ind in indexes
            @constraint(model,w[ind,:].==0)
        end
        @objective(model,Min,sum(w[i] for i in 1:r))
        optimize!(model)
        answer[:,j]= JuMP.value.(v)
    end
    return answer
end

solv (generic function with 1 method)

In [15]:
answer=[]
indexes= zero_rows(answer)
answer= solv(indexes)
length(zero_rows(answer))

3646

# writing to output file

In [28]:
using DelimitedFiles
open("output.txt","w") do io
    writedlm(io, answer,",")
end